In [ ]:
from numpy import exp, array, random, dot

# Define the training set inputs and outputs
training_set_inputs = array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
training_set_outputs = array([[0, 1, 1, 0]]).T

# Seed the random number generator for reproducibility
random.seed(1)

# Initialize synaptic weights randomly with mean 0
synaptic_weights = 2 * random.random((3, 1)) - 1

# Train the neural network
for iteration in range(10000):
    # Calculate the output using the sigmoid function
    output = 1 / (1 + exp(-(dot(training_set_inputs, synaptic_weights))))

    # Adjust weights using gradient descent
    synaptic_weights += dot(training_set_inputs.T, (training_set_outputs - output) * output * (1 - output))

# Test the neural network with a new situation
test_input = array([1, 0, 0])
test_output = 1 / (1 + exp(-(dot(test_input, synaptic_weights))))
print(f"Test input: {test_input}, Predicted output: {test_output}")

[0.99993704]


In [53]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the autoencoder
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(256, 12),
            nn.Sigmoid()
        )
        self.decoder = nn.Sequential(
            nn.Linear(12, 256),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Prepare the dataset (normalize to [0, 1] for Sigmoid activation)
data = torch.eye(256) # * 0.99 + 0.01

# Initialize the model, loss function, and optimizer
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the autoencoder
epochs = 50000
for epoch in range(epochs):
    optimizer.zero_grad()
    encoded, decoded = model(data)
    loss = criterion(decoded, data)
    loss.backward()
    optimizer.step()

    if loss.item() < 0.0001:
        print(f"Early stopping at epoch {epoch + 1}, Loss: {loss.item():.6f}")
        break

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.6f}")

# Inspect the middle layer (encoded representation)
with torch.no_grad():
    encoded, _ = model(data)
    print("Encoded representations:")
    print(encoded)
    
# Demonstrate the operation of the encoder
with torch.no_grad():
    good = 0
    print('  0', end=' ')
    for i in range(256):  # Show the first 10 one-hot encoded numbers
        input_vector = data[i].unsqueeze(0)  # Select one-hot encoded vector
        enc, dec = model(input_vector)  # Pass through the encoder
       
        out = dec.numpy().argmax()
        
        
        if i == out:
            good += 1
            print('+', end='')
        else:
            print(' ', end='')
       
        if (i+1)  % 32 == 0:
            print(f'\n{i+1:3d} ', end='')
       
    print(f'\nGood: {good} / 256')
    

Epoch [1000/50000], Loss: 0.003903
Epoch [2000/50000], Loss: 0.003891
Epoch [3000/50000], Loss: 0.003890
Epoch [4000/50000], Loss: 0.003889
Epoch [5000/50000], Loss: 0.003873
Epoch [6000/50000], Loss: 0.003440
Epoch [7000/50000], Loss: 0.002521
Epoch [8000/50000], Loss: 0.001162
Epoch [9000/50000], Loss: 0.000518
Epoch [10000/50000], Loss: 0.000398
Epoch [11000/50000], Loss: 0.000287
Epoch [12000/50000], Loss: 0.000250
Epoch [13000/50000], Loss: 0.000202
Epoch [14000/50000], Loss: 0.000183
Epoch [15000/50000], Loss: 0.000141
Epoch [16000/50000], Loss: 0.000123
Epoch [17000/50000], Loss: 0.000123
Epoch [18000/50000], Loss: 0.000122
Epoch [19000/50000], Loss: 0.000122
Epoch [20000/50000], Loss: 0.000122
Epoch [21000/50000], Loss: 0.000107
Epoch [22000/50000], Loss: 0.000107
Epoch [23000/50000], Loss: 0.000107
Epoch [24000/50000], Loss: 0.000107
Epoch [25000/50000], Loss: 0.000107
Early stopping at epoch 25541, Loss: 0.000099
Encoded representations:
tensor([[9.9645e-01, 2.4882e-01, 4.225

In [54]:
i =  200
input_vector = data[i].unsqueeze(0)  # Select one-hot encoded vector
enc, dec = model(input_vector)  # Pass through the encoder

with torch.no_grad():
    enc = enc.numpy()

[ round(e) for e in enc[0]]

[0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0]